In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Dirs - must be absolute paths!
LOG_DIR = '/tmp/tf/mnist_logistic_regression'
MNIST_DIR = "/home/tkornuta/data/mnist"

# Set learning parameters.
LEARNING_RATE = 1e-4
BATCH_SIZE = 100
N_EPOCHS = 10

### A. Import MNIST datset, use one-hot encoding for labels.

In [2]:
mnist_dataset = input_data.read_data_sets(MNIST_DIR, one_hot=True)

Extracting /home/tkornuta/data/mnist/train-images-idx3-ubyte.gz
Extracting /home/tkornuta/data/mnist/train-labels-idx1-ubyte.gz
Extracting /home/tkornuta/data/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/tkornuta/data/mnist/t10k-labels-idx1-ubyte.gz


### B. Define the tensor graph.

In [3]:
# Reset graph - just in case.
tf.reset_default_graph()

# B. 
# 0. Placeholders for inputs.
with tf.name_scope("Input_data"):
    # Shape - none, as we will feed both training batches as well as test datasets.
    x = tf.placeholder(tf.float32, shape=None, name="x")
    targets = tf.placeholder(tf.float32, shape=None, name="target")
with tf.name_scope("Input_visualization") as scope:
    x_image = tf.reshape(x, [-1,28,28,1])
    image_summ = tf.summary.image("Example_images", x_image)

# 1. Inference ops.
with tf.name_scope("Inference"):
    w = tf.Variable(tf.random_normal(shape=[784,10], stddev=0.01), name="weights") 
    b = tf.Variable(tf.zeros(shape=[1,10]), name="bias")
    logits = tf.add(tf.matmul(x,w), b, name="logits")
    # Add histograms to TensorBoard.
    w_hist = tf.summary.histogram("w", w)
    b_hist = tf.summary.histogram("b", b)
with tf.name_scope("Activation_visualization"):
    # Interate through neurons.
    for output_neuron in range(10):
        w_activ = tf.slice(w, [0,output_neuron], [784,1])
        w_act_reshaped = tf.reshape(w_activ, [1,28,28,1])
        #  # Get activations from consecutive cols.
        image_summ = tf.summary.image("w_activations", w_act_reshaped)

# 2. Loss ops.
with tf.name_scope("Loss"):
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=targets)
    # Loss = mean over examples in the batch.
    loss = tf.reduce_mean(entropy)
    # Add loss summary.
    loss_summary = tf.summary.scalar("loss", loss)
    
# 3. Training ops.  
with tf.name_scope("Training"):
    optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(loss)
with tf.name_scope("Evaluating") as scope:
    # Count correct predictions by a simple argmax trick on each sample in a batch.
    correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(targets,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    # Add accuracy summary.
    accuracy_summary = tf.summary.scalar("accuracy", accuracy)

# Merge all summaries.
summaries = tf.summary.merge_all()

# 4. Init global variable.
init = tf.global_variables_initializer()

#### Helper functions

In [4]:
def feed_dict(dataset):
  """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
  if dataset=="train":
    xs, ys = mnist_dataset.train.next_batch(BATCH_SIZE)
  elif dataset=="valid":
    xs, ys = mnist_dataset.validation.images, mnist_dataset.validation.labels
  else: # test
    xs, ys = mnist_dataset.test.images, mnist_dataset.test.labels
  return {x: xs, targets: ys}

### C. Run session.

In [5]:
# Eventually clear the log dir.
if tf.gfile.Exists(LOG_DIR):
  tf.gfile.DeleteRecursively(LOG_DIR)
# Create (new) log dir.
tf.gfile.MakeDirs(LOG_DIR)

In [6]:
# Create session.
sess = tf.InteractiveSession()
# Merge all the summaries and write them out to /tmp/mnist_logs (by default)
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(LOG_DIR + '/train', sess.graph)
valid_writer = tf.summary.FileWriter(LOG_DIR + '/valid')
test_writer = tf.summary.FileWriter(LOG_DIR + '/test')

# Initialize variables.
#tf.global_variables_initializer().run()
sess.run(init)

# First, check accuracy on test dataset.
#summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict("test"))
#test_writer.add_summary(summary, 0)
#print('Initial accuracy on test set: %s' % (acc))

# Perform training with minibatches and validation every 100 training steps.
n_batches = int(mnist_dataset.train.num_examples/BATCH_SIZE)
for e in range (N_EPOCHS):
  for b in range (n_batches):
    if b % 100 == 0:  # Record summaries and valid-set accuracy
      summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict("valid"))
      valid_writer.add_summary(summary, b+e*n_batches)
      valid_writer.flush()
      print('Accuracy at step %s: %s' % (b+e*n_batches, acc))
    else:  # Record train set summaries, and train
      summary, acc = sess.run([merged, optimizer], feed_dict=feed_dict("train"))
      train_writer.add_summary(summary, b+e*n_batches)
      train_writer.flush()

# Finally, check accuray on test dataset
summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict("test"))
test_writer.add_summary(summary, n_batches*N_EPOCHS)
print('Final accuracy on test set: %s' % (acc))

# Close writers and session.
train_writer.close()
valid_writer.close()
test_writer.flush()
test_writer.close()
sess.close()

Accuracy at step 0: 0.1276
Accuracy at step 100: 0.6598
Accuracy at step 200: 0.7658
Accuracy at step 300: 0.7892
Accuracy at step 400: 0.8084
Accuracy at step 500: 0.8144
Accuracy at step 550: 0.8164
Accuracy at step 650: 0.8296
Accuracy at step 750: 0.8358
Accuracy at step 850: 0.8408
Accuracy at step 950: 0.8488
Accuracy at step 1050: 0.8516
Accuracy at step 1100: 0.8556
Accuracy at step 1200: 0.8586
Accuracy at step 1300: 0.8628
Accuracy at step 1400: 0.8652
Accuracy at step 1500: 0.8672
Accuracy at step 1600: 0.8712
Accuracy at step 1650: 0.8722
Accuracy at step 1750: 0.8738
Accuracy at step 1850: 0.8762
Accuracy at step 1950: 0.8794
Accuracy at step 2050: 0.8808
Accuracy at step 2150: 0.883
Accuracy at step 2200: 0.8852
Accuracy at step 2300: 0.8852
Accuracy at step 2400: 0.8858
Accuracy at step 2500: 0.8886
Accuracy at step 2600: 0.8896
Accuracy at step 2700: 0.8926
Accuracy at step 2750: 0.8926
Accuracy at step 2850: 0.8942
Accuracy at step 2950: 0.8958
Accuracy at step 3050: 0